<a href="https://colab.research.google.com/github/gnozadi/Cow-detection-segmentation/blob/main/detection/yolo6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download MT-YOLOv6 repository and install requirements
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6
!pip install -r requirements.txt

Cloning into 'YOLOv6'...
remote: Enumerating objects: 3653, done.
remote: Counting objects: 100% (1208/1208), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 3653 (delta 1078), reused 1010 (delta 1001), pack-reused 2445
Receiving objects: 100% (3653/3653), 43.34 MiB | 21.89 MiB/s, done.
Resolving deltas: 100% (2186/2186), done.
/content/YOLOv6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 102.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.2

In [2]:
import cv2
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def resize_input_image(d_path, size):
  image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
  imgs_list = [filename for filename in os.listdir(d_path) if os.path.splitext(filename)[-1] in image_extensions]
  for i, f in enumerate(imgs_list):
    img = cv2.imread(d_path+'/'+f)
    out=cv2.resize(img,(size,size),interpolation= cv2.INTER_LINEAR)
    cv2.imwrite(d_path+'/'+f, out)

In [5]:

all_paths = [
    "/content/drive/MyDrive/cow-dataset/train/images",
    "/content/drive/MyDrive/cow-dataset/test/images",
    "/content/drive/MyDrive/cow-dataset/valid/images"
]

for p in all_paths:
  resize_input_image(p, 640)


In [6]:
!python tools/train.py --batch 32 --conf configs/yolov6s.py --epochs 50 --img-size 640 --data /content/drive/MyDrive/cow-dataset/data.yaml --device 0

2023-08-20 10:59:18.546036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 10:59:19.435575: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using 1 GPU for training... 
training args are: Namespace(data_path='/content/drive/MyDrive/cow-dataset/data.yaml', conf_file='configs/yolov6s.py', img_size=640, rect=False, batch_size=32, epochs=50, workers=8, device='0', eval_interval=20, eval_final_only=False, heavy_eval_range=50, check_images=False, check_labels=False, output_dir='./runs/train', name='exp', dist_url='env://', gpu_count=0, local_rank=-1, resume=False, write_trainbatch_tb=False, stop_aug_last_n_epoch=15, save_ckpt_on_last_n_epoch=-1, distill=False, distill_feat=False, quant=False, 

In [7]:
!cp -r "runs/train/exp" "/content/drive/MyDrive/YOLO6_Results"

In [8]:
!python tools/eval.py --data /content/drive/MyDrive/cow-dataset/data.yaml --img-size 640 --weights runs/train/exp/weights/best_ckpt.pt --device 0


Namespace(data='/content/drive/MyDrive/cow-dataset/data.yaml', weights='runs/train/exp/weights/best_ckpt.pt', batch_size=32, img_size=640, conf_thres=0.03, iou_thres=0.65, task='val', device='0', half=False, save_dir='runs/val/', name='exp', shrink_size=0, infer_on_rect=True, reproduce_640_eval=False, eval_config_file='./configs/experiment/eval_640_repro.py', do_coco_metric=True, do_pr_metric=False, plot_curve=True, plot_confusion_matrix=False, verbose=False, config_file='', specific_shape=False, height=None, width=None)
Loading checkpoint from runs/train/exp/weights/best_ckpt.pt

Fusing model...
Switch model to deploy modality.
Model Summary: Params: 18.51M, Gflops: 45.18
img record infomation path is:/content/drive/MyDrive/cow-dataset/valid/.images_cache.json
Val: Checking formats of labels with 2 process(es): 
174 label(s) found, 0 label(s) missing, 0 label(s) empty, 0 invalid label files: 100% 174/174 [00:00<00:00, 784.62it/s]
Convert to COCO format
100% 174/174 [00:00<00:00, 44495